# Imports 

In [1]:
import pandas as pd
import numpy as np
import torch
from torch.nn import functional as F
import scipy
import sklearn
from umap import UMAP

import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from netcal.metrics import ECE, MCE
from netcal.scaling import TemperatureScaling
from netcal.presentation import ReliabilityDiagram
from sklearn.metrics import brier_score_loss
from sklearn.preprocessing import OneHotEncoder

# Analysis Functions 

In [2]:
def test_report(l, confidence=0.95):
    for df in l:
        df.columns = list(df.columns[1:]) + ['model_name']
        df['accuracy'] = 100 * df['accuracy']
        a = df['accuracy'].to_numpy()
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        print(m, h)

In [3]:
def conf_int(data, confidence=0.95):
    a = data.to_numpy()
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

In [4]:
def prob_metrics(df):
    df.columns = ['task', '0', '1', '2', '3', '4', 'actual']
    df[['0', '1', '2', '3', '4']] = F.softmax(torch.tensor(
        df[['0', '1', '2', '3', '4']].to_numpy()), dim=1).numpy()
    
    # Calculate ECE, MCE
    n_bins = 10
    confidences = df[['0', '1', '2', '3', '4']].to_numpy()
    ground_truth = df['actual'].to_numpy()
    ece = ECE(n_bins)
    mce = MCE(n_bins)
    uncalibrated_score_ece = ece.measure(confidences, ground_truth)
    temperature = TemperatureScaling()
    temperature.fit(confidences, ground_truth)
    calibrated = temperature.transform(confidences)
    calibrated_score_ece = ece.measure(calibrated, ground_truth)
    
    uncalibrated_score_mce = mce.measure(confidences, ground_truth)
    calibrated_score_mce = mce.measure(calibrated, ground_truth)

    # Reliability diagram
    diagram = ReliabilityDiagram(n_bins)
    diagram.plot(calibrated, ground_truth)
    
    return calibrated_score_ece, calibrated_score_mce

In [5]:
def report(df, tasks, confidence=0.95):
    df.columns = list(df.columns[1:]) + ['model_name']
    df['accuracy'] = 100 * df['accuracy']
    a = df['accuracy'].to_numpy()
    a = np.split(a, len(a)/tasks, axis=0)
    for accs in a:
        n = len(accs)
        m, se = np.mean(accs), scipy.stats.sem(accs)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        print(m, h)

In [368]:
def plot_summarize(df_train, df_validate, vis_columns, meta_batch_size):
    df_valid.columns = list(df_valid.columns[1:]) + ['model_name'] # model name is at the end of the column "hacks"
    
    def process_df(df, vis_columns):
        df = df[vis_columns]
        df.rename_axis('iteration').reset_index()
        df = df.assign(accuracy = df["accuracy"]*100) if "accuracy" in df.columns else df
        return df
    
    df_train = process_df(df_train, vis_columns)
    df_validate = process_df(df_validate, vis_columns)
    
    df_train = df_train.groupby(np.arange(len(df_train)) // meta_batch_size).mean() # produce results per meta batch
    df_train.rename_axis('iteration').reset_index()
    
    if "accuracy" in df_validate.columns:
        iteration_idx = df_validate['accuracy'].idxmax() 
        print('Validation Metrics Stats of Best Model at {}-th (max) Iteration'.format(iteration_idx))
        display(df_validate.describe())
    
    # Plot train and validate metrics
    fig = px.line(df_train, title="Training Metrics")
    fig.show()

    fig = px.line(df_validate, title="Validation Metrics")
    fig.show()
    

# Analysing runs from CSV's

In [369]:
df_train = pd.read_csv(f'./output/TRIDENT_miniimagenet_5-way_1-shot_1-queries/exp1/train.csv')
df_valid = pd.read_csv(f'./output/TRIDENT_valid_miniimagenet_5-way_1-shot_1-queries/exp1/valid.csv')
df_test = pd.read_csv(f'./output/TRIDENT_test_miniimagenet_5-way_1-shot_1-queries/exp1/test.csv')
# df_test = df_test[~(df_test['CE_Loss'] == 'model_last.pt')]
# df_valid = df_valid[~(df_valid['CE_Loss'] == 'model_last.pt')]

In [370]:
# columns = ['accuracy', 'ELBO', 'Label_KL', 'Style_KL', 'Reconst_Loss', 'CE_Loss'] #Choose from these
columns = ['accuracy']
plot_summarize(df_train, df_valid, columns, 20)

Validation Metrics Stats of Best Model at 59-th (max) Iteration


,accuracy
count,500.000000
mean,20.520000
std,16.525004
min,0.000000
25%,0.000000
50%,20.000000
75%,40.000001
max,80.000001


In [343]:
df_test = pd.read_csv(f'./output/TRIDENT_test_miniimagenet_5-way_1-shot_1-queries/exp1/test.csv')
test_report([df_test]) # Can also run test.py multiple times for multiple test.csv's and add in the argument list  

21.400000398357708 1.3822186661782636


# Prob Metrics 

In [18]:
df_preds = pd.read_csv('PATH TO preds.csv')
ece, mce = prob_metrics(df_preds)
print(ece,mce)

FileNotFoundError: [Errno 2] No such file or directory: 'PATH TO preds.csv'

In [ ]:
def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))
encoder = OneHotEncoder(sparse=False)
onehot = encoder.fit_transform(np.array(df_preds['actual']).reshape(-1, 1))
brier_multi(onehot, df_preds[['0','1','2','3','4']])

# Visualizing the Latents 

In [ ]:
latents0 = torch.load('PATH to a random latents_0')
latents = torch.load('PATH to a random latents')

In [ ]:
# Making Before and After latent-datasets 
df0 = pd.DataFrame(np.array(latents0['label_latents'][0]))
df0['class'] = pd.Series(list(np.full((10,), 0)) + list(np.full((10,), 1)) + list(
    np.full((10,), 2)) + list(np.full((10,), 3)) + list(np.full((10,), 4)))
features0 = df0.iloc[:, :-1]
df = pd.DataFrame(np.array(latents['label_latents'][0]))
df['class'] = pd.Series(list(np.full((10,), 0)) + list(np.full((10,), 1)) + list(
    np.full((10,), 2)) + list(np.full((10,), 3)) + list(np.full((10,), 4)))
features = df.iloc[:, :-1]

# UMAP projection to 2D space 
umap = UMAP(n_components=2, init='random', random_state=0)
proj_2d0 = umap.fit_transform(features0)
umap = UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap.fit_transform(features)

In [ ]:
sklearn.metrics.davies_bouldin_score(proj_2d0, df['class'])

In [ ]:
sklearn.metrics.davies_bouldin_score(proj_2d, df['class'])

In [ ]:
plt.figure(figsize=(25, 12))
sns.relplot(x=proj_2d[:, 0], y=proj_2d[:, 1], hue=df['class'].astype(
    int), palette="Dark2", style=df['class'].astype(int), s=250, legend=False)
a=sns.kdeplot(x=proj_2d[:, 0], y=proj_2d[:, 1],
            hue=df['class'].astype(int), palette="Pastel2", legend=False)
sns.despine(right=True)

In [ ]:
plt.figure(figsize=(10, 6))
sns.relplot(x=proj_2d0[:, 0], y=proj_2d0[:, 1], hue=df['class'].astype(
    int), palette="Dark2", style=df['class'].astype(int), s=250, legend=False)
a=sns.kdeplot(x=proj_2d0[:, 0], y=proj_2d0[:, 1],
            hue=df['class'].astype(int), palette="Pastel2", legend=False)
sns.despine(right=True)

In [ ]:
sfig = a.get_figure()
sfig.savefig('label_latents0.jpeg', dpi=1000)

In [ ]:
# Making Before and After latent-datasets 
df0 = pd.DataFrame(np.array(latents0['semantic_latents'][0]))
df0['class'] = pd.Series(list(np.full((10,), 0)) + list(np.full((10,), 1)) + list(
    np.full((10,), 2)) + list(np.full((10,), 3)) + list(np.full((10,), 4)))
features0 = df0.iloc[:, :-1]
df = pd.DataFrame(np.array(latents['semantic_latents'][0]))
df['class'] = pd.Series(list(np.full((10,), 0)) + list(np.full((10,), 1)) + list(
    np.full((10,), 2)) + list(np.full((10,), 3)) + list(np.full((10,), 4)))
features = df.iloc[:, :-1]

# UMAP projection to 2D space
umap = UMAP(n_components=2, init='random', random_state=0)
proj_2d0 = umap.fit_transform(features0)
umap = UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap.fit_transform(features)

In [ ]:
sklearn.metrics.davies_bouldin_score(proj_2d0, df['class'])

In [ ]:
sklearn.metrics.davies_bouldin_score(proj_2d, df['class'])

In [ ]:
plt.figure(figsize=(10, 6))
sns.relplot(x=proj_2d[:, 0], y=proj_2d[:, 1], hue=df['class'].astype(
    int), palette="Dark2", style=df['class'].astype(int), s=250, legend=False)
a=sns.kdeplot(x=proj_2d[:, 0], y=proj_2d[:, 1],
            hue=df['class'].astype(int), palette="Pastel2", legend=False)
sns.despine(right=True)

In [ ]:
#plt.figure(figsize=(10, 6))
sns.relplot(x=proj_2d0[:, 0], y=proj_2d0[:, 1], hue=df['class'].astype(
    int), palette="Dark2", style=df['class'].astype(int), s=250, legend=False)
a= sns.kdeplot(x=proj_2d0[:, 0], y=proj_2d0[:, 1],
            hue=df['class'].astype(int), palette="Pastel2", legend=False)
sns.despine(right=True)

In [ ]:
sfig = a.get_figure()
sfig.savefig('style_latents0.jpeg', dpi=1000)